Sample data set generated from the [Databricks Data Generator](https://databrickslabs.github.io/dbldatagen/public_docs/index.html)

In [0]:
%pip install dbldatagen

Python interpreter will be restarted.
  Using cached dbldatagen-0.3.0-py3-none-any.whl (69 kB)
Python interpreter will be restarted.


In [0]:
import random
hash = random.getrandbits(128)

BASE_PATH = '/tmp/dml_sample/merge/' + str(hash) + '/'

dbutils.fs.mkdirs(BASE_PATH)
customers_location = BASE_PATH + "customers"

print(customers_location)

/tmp/dml_sample/merge/97392762246464809283238622335320419923/customers


In [0]:
# Number of Rows of customer data
data_rows = 10000000
change_set_size = 10000

In [0]:
import dbldatagen as dg
import pyspark.sql.functions as F

spark.catalog.clearCache()
shuffle_partitions_requested = 8
partitions_requested = 32

spark.conf.set("spark.sql.shuffle.partitions", shuffle_partitions_requested)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 20000)

dataspec = (dg.DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
            .withColumn("customer_id","long", uniqueValues=data_rows)
            .withColumn("name", percentNulls=0.01, template=r'\\w \\w|\\w a. \\w')
            .withColumn("alias", percentNulls=0.01, template=r'\\w \\w|\\w a. \\w')
            .withColumn("payment_instrument_type", values=['paypal', 'Visa', 'Mastercard',
                        'American Express', 'discover', 'branded visa', 'branded mastercard'],
                        random=True, distribution="normal")
            .withColumn("int_payment_instrument", "int",  minValue=0000, maxValue=9999,  baseColumn="customer_id",
                        baseColumnType="hash", omit=True)
            .withColumn("payment_instrument", expr="format_number(int_payment_instrument, '**** ****** *####')",
                        baseColumn="int_payment_instrument")
            .withColumn("email", template=r'\\w.\\w@\\w.com|\\w-\\w@\\w')
            .withColumn("customer_notes", text=dg.ILText(words=(1,8)))
            .withColumn("created_ts", "timestamp", expr="now()")
            .withColumn("modified_ts", "timestamp", expr="now()")
            .withColumn("memo", expr="'original data'")
            )
df1 = dataspec.build()

# write table
df1.write.format("delta").save(customers_location)

INFO: Version : VersionInfo(major='0', minor='3', patch='0', release='', build='')


In [0]:
tableDefn=dataspec.scriptTable(name="customers", location=customers_location)
spark.sql(tableDefn)
tableDefn

Out[4]: "CREATE TABLE IF NOT EXISTS customers (\n    customer_id bigint,\n    name string,\n    alias string,\n    payment_instrument_type string,\n    payment_instrument string,\n    email string,\n    customer_notes string,\n    created_ts timestamp,\n    modified_ts timestamp,\n    memo string\n)\nusing delta\nlocation '/tmp/dml_sample/merge/97392762246464809283238622335320419923/customers'"

In [0]:
%sql
-- lets check our table

SELECT * FROM customers

customer_id,name,alias,payment_instrument_type,payment_instrument,email,customer_notes,created_ts,modified_ts,memo
1,ut m. consectetur,ad z. irure,American Express,**** ****** *669,officia-exercitation@voluptate,Incididunt magna in in quis eiusmod anim occaecat.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
2,laborum ullamco,cupidatat ullamco,discover,**** ****** *2728,in.qui@duis.com,Culpa adipiscing cillum.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
3,duis e. laboris,laborum quis,American Express,**** ****** *707,sed.sunt@in.com,Duis.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
4,reprehenderit ea,aute amet,American Express,**** ****** *3940,tempor.nulla@occaecat.com,Aute eiusmod cillum excepteur sint pariatur irure commodo.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
5,ut amet,non sed,discover,**** ****** *4268,ex.officia@tempor.com,Laborum duis labore sint quis.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
6,ad l. ad,occaecat amet,discover,**** ****** *5445,nostrud.non@nostrud.com,Velit duis eiusmod.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
7,ut l. fugiat,adipiscing adipiscing,American Express,**** ****** *6811,dolore-sed@esse,Eu.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
8,consectetur h. commodo,ut f. aliqua,discover,**** ****** *5916,ad.ullamco@ea.com,Enim.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
9,dolor m. mollit,commodo v. ex,Mastercard,**** ****** *3649,anim-dolore@duis,Cillum exercitation consectetur.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data
10,pariatur g. velit,sit ut,Mastercard,**** ****** *5677,ex-in@voluptate,Tempor anim minim ex dolor velit nulla culpa.,2023-02-03T18:38:41.445+0000,2023-02-03T18:38:41.445+0000,original data


In [0]:
import pandas as pd
  
# list of strings
lst = dbutils.fs.ls(customers_location)
  
# Calling DataFrame constructor on list
before_merge_df = pd.DataFrame(lst)
before_merge_df = before_merge_df.loc[before_merge_df["name"] != "_delta_log/"]
before_merge_df

,path,name,size,modificationTime
1,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00000-2ebf808f-0d4b-41de-9e5c-4765e11fc0c...,102050498,1675449813000
2,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00001-698ee55e-7596-44bb-93bc-a4dd37412bc...,102049939,1675449811000
3,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00002-d3f7e97c-332c-4502-aa6b-7669dd4f9dd...,102051544,1675449806000
4,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00003-2c44f562-2776-4c07-9860-0d57aad1a3b...,102044144,1675449812000
5,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00004-6cdb7534-7bc2-4f10-a0c6-4002414a0b1...,102045564,1675449810000
6,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00005-85c7515e-da52-44b0-9c21-00439eaf644...,102047042,1675449807000
7,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00006-e7e908de-5aba-4849-af28-685ca14619b...,102047418,1675449817000
8,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00007-d24a7d9a-b4bb-476f-ace2-0234b778249...,102047029,1675449808000
9,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00008-20fc2a05-4154-43c8-b59c-8d85959c5d7...,102043503,1675449809000
10,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00009-4cb39547-0716-4bff-a535-65c77e8e615...,102053577,1675449815000


In [0]:
import dbldatagen as dg
import pyspark.sql.functions as F

start_of_new_ids = df1.select(F.max('customer_id')+1).collect()[0][0]

print(start_of_new_ids)

df1_inserts = (dataspec.clone()
        .option("startingId", start_of_new_ids)
        .withRowCount(round(change_set_size * 0.2))
        .build()
        .withColumn("memo", F.lit("insert"))
        .withColumn("customer_id", F.expr(f"customer_id + {start_of_new_ids}"))
              )

# read the written data - if we simply recompute, timestamps of original will be lost
df_original = spark.read.format("delta").load(customers_location)

df1_updates = (df_original.sample(False, 0.1)
        .limit(round(change_set_size * 0.8))
        .withColumn("alias", F.lit('modified alias'))
        .withColumn("modified_ts",F.expr('now()'))
        .withColumn("memo", F.lit("update")))

df_changes = df1_inserts.union(df1_updates)

# randomize ordering
df_changes = (df_changes.withColumn("order_rand", F.expr("rand()"))
              .orderBy("order_rand")
              .drop("order_rand")
              )


display(df_changes)

100000001


customer_id,name,alias,payment_instrument_type,payment_instrument,email,customer_notes,created_ts,modified_ts,memo
81302807,cillum g. magna,modified alias,American Express,**** ****** *6458,ut.irure@qui.com,Et nisi sit.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81321574,sit m. ex,modified alias,American Express,**** ****** *8710,reprehenderit.ut@esse.com,Laboris ut cupidatat non in velit.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81301861,ut aliquip,modified alias,discover,**** ****** *6811,ipsum.enim@consequat.com,Fugiat aute sint eiusmod minim consectetur non reprehenderit.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81315706,officia eiusmod,modified alias,discover,**** ****** *774,deserunt.velit@cupidatat.com,Veniam nostrud.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81254838,cillum s. in,modified alias,Visa,**** ****** *9921,nostrud-eiusmod@ex,Cupidatat consequat magna id non.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81268657,magna e. incididunt,modified alias,American Express,**** ****** *3189,cillum-incididunt@officia,Consequat cupidatat qui commodo minim excepteur aliquip laborum.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81269695,nulla culpa,modified alias,discover,**** ****** *860,fugiat-pariatur@exercitation,In nisi.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81264183,dolor ullamco,modified alias,American Express,**** ****** *1795,reprehenderit-excepteur@ullamco,Veniam occaecat dolor excepteur velit et id ipsum.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81295976,non n. ut,modified alias,discover,**** ****** *6047,eu-excepteur@labore,Ad dolor aute consequat.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update
81306631,amet ut,modified alias,American Express,**** ****** *4615,aute.incididunt@lorem.com,Tempor nisi sunt labore in mollit.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:18.078+0000,update


In [0]:
df_changes.createOrReplaceTempView("change_set")

In [0]:
%sql

SELECT memo, count(memo)
FROM change_set
GROUP BY memo

memo,count(memo)
insert,2000
update,8000


In [0]:
df_changes.dropDuplicates(["customer_id"]).createOrReplaceTempView("customers1_changes")
sqlStmt = dataspec.scriptMerge(tgtName="customers", srcName="changes",
                               joinExpr="src.customer_id=tgt.customer_id",
                               updateColumns=["alias", "memo","modified_ts"],
                               updateColumnExprs=[ ("memo", "'updated on merge'"),
                                                   ("modified_ts", "now()")
                                                 ])

print(sqlStmt)

MERGE INTO `customers` as tgt
USING `changes` as src
ON src.customer_id=tgt.customer_id
WHEN MATCHED THEN UPDATE  SET alias=src.alias, memo='updated on merge', modified_ts=now()
WHEN NOT MATCHED THEN INSERT (customer_id,name,alias,payment_instrument_type,payment_instrument,email,customer_notes,created_ts,modified_ts,memo) VALUES (src.customer_id, src.name, src.alias, src.payment_instrument_type, src.payment_instrument, src.email, src.customer_notes, src.created_ts, src.modified_ts, src.memo)


In [0]:
%sql

MERGE INTO `customers` as tgt
  USING `change_set` as src
  ON src.customer_id=tgt.customer_id
  WHEN MATCHED THEN 
    UPDATE SET alias=src.alias, memo='updated on merge', modified_ts=now()
  WHEN NOT MATCHED THEN 
    INSERT (customer_id,name,alias,payment_instrument_type,payment_instrument,email,customer_notes,created_ts,modified_ts,memo) VALUES (src.customer_id, src.name, src.alias, src.payment_instrument_type, src.payment_instrument, src.email, src.customer_notes, src.created_ts, src.modified_ts, src.memo)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10000,8000,0,2000


For a small change data set relative to your source files, the change set is isolated to a single smaller file.  This allows the ordering and previous files to stay the same.  No major shuffling of pre-existing files. 

For a larger change data set, there is more shuffling and files that are added.  May require an OPTIMIZE command to be executed.

In [0]:
# list of strings
lst = dbutils.fs.ls(customers_location)
  
# Calling DataFrame constructor on list
after_merge_df = pd.DataFrame(lst)
after_merge_df = after_merge_df.loc[after_merge_df["name"] != "_delta_log/"]
after_merge_df

,path,name,size,modificationTime
1,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00000-2ebf808f-0d4b-41de-9e5c-4765e11fc0c...,102050498,1675449813000
2,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00000-45e5e709-e8e7-4ce2-871d-021e49c29a0...,33480383,1675450422000
3,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00000-e96d0746-2ff5-4bed-98a8-806b5d62fa1...,389465,1675450414000
4,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00001-2a9c201d-1502-40d6-8045-0500867b97c...,33481920,1675450422000
5,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00001-698ee55e-7596-44bb-93bc-a4dd37412bc...,102049939,1675449811000
6,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00002-126c0e0a-dcfd-4ac9-8923-a2ae59e1b18...,33476068,1675450420000
7,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00002-d3f7e97c-332c-4502-aa6b-7669dd4f9dd...,102051544,1675449806000
8,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00003-2c44f562-2776-4c07-9860-0d57aad1a3b...,102044144,1675449812000
9,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00004-6cdb7534-7bc2-4f10-a0c6-4002414a0b1...,102045564,1675449810000
10,dbfs:/tmp/dml_sample/merge/9739276224646480928...,part-00005-85c7515e-da52-44b0-9c21-00439eaf644...,102047042,1675449807000


In [0]:
%sql

SELECT COUNT(*)
FROM CUSTOMERS


count(1)
100002000


In [0]:
%sql

SELECT *
FROM CUSTOMERS
WHERE memo = "updated on merge"


customer_id,name,alias,payment_instrument_type,payment_instrument,email,customer_notes,created_ts,modified_ts,memo
28129169,adipiscing z. officia,modified alias,American Express,**** ****** *2456,pariatur.in@cillum.com,Lorem.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28198046,laborum a. deserunt,modified alias,American Express,**** ****** *8560,magna-ea@ea,Ex eu sed veniam veniam aliquip occaecat esse.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28192956,consectetur e. amet,modified alias,discover,**** ****** *9424,fugiat-excepteur@officia,Sit.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28125801,dolore a. aute,modified alias,American Express,**** ****** *8424,proident-cillum@non,Pariatur consectetur ipsum do in.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28156126,deserunt enim,modified alias,American Express,**** ****** *4248,duis.eu@pariatur.com,Qui aliqua nostrud labore eiusmod esse.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28183660,commodo sit,modified alias,American Express,**** ****** *1392,duis-ad@dolore,Sunt in officia mollit eu culpa ipsum.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28195745,deserunt reprehenderit,modified alias,American Express,**** ****** *4736,amet.proident@pariatur.com,Sit in officia.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28169503,voluptate sint,modified alias,Mastercard,**** ****** *2400,aute-labore@duis,Fugiat qui eiusmod esse anim voluptate.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28182782,culpa sit,modified alias,American Express,**** ****** *6208,ipsum-lorem@cupidatat,Eiusmod do ut ut elit magna labore ut.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge
28190735,sunt ut,modified alias,discover,**** ****** *2840,laborum-dolore@irure,In.,2023-02-03T18:38:41.445+0000,2023-02-03T18:53:28.525+0000,updated on merge


In [0]:
%sql 

DROP TABLE customers

In [0]:
dbutils.fs.rm(customers_location, True)

Out[10]: True